In [215]:
import cadquery
import pickle
import cadquery as cq
from io import StringIO
import contextlib
from openai import OpenAI
import os

In [73]:
data_path = "data_complex.pkl"

with open(data_path, "rb") as f:
    dataset = pickle.load(f)

In [ ]:
def execute_cadquery_code(code_string):
    safe_globals = {"cq": cq}

    output = StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(code_string, safe_globals)

            if "r" in safe_globals and isinstance(safe_globals["r"], cq.Workplane):
                return safe_globals["r"], output.getvalue()

        return None, "No valid CadQuery object named 'r' found"
    except Exception as e:
        return None, f"Error: {str(e)}"

In [138]:
idx = 1

code, description = dataset[idx]["code"], dataset[idx]["description"]

model = execute_cadquery_code(code)[0]

print(f"##Code for index {idx}:\n{code}\n\n##Description:\n{description}\n")
# model, _ = execute_cadquery_code(dataset[indices[idx]]["code"])
model

##Code for index 1:
import cadquery as cq
w0=cq.Workplane('YZ',origin=(-100,-67,-83))
w1=cq.Workplane('XY',origin=(37,-67,-83))
r=w0.sketch().face(w0.sketch().segment((0,0),(133,0)).segment((133,0),(133,167)).segment((133,167),(67,167)).segment((67,167),(67,67)).segment((67,67),(0,67)).segment((0,67),(0,0)).assemble()).finalize().extrude(200).cut(w1.sketch().face(w1.sketch().segment((0,0),(67,0)).segment((67,0),(67,67)).segment((67,67),(0,0)).assemble()).finalize().extrude(133))

##Description:
In the first part, create a new coordinate system with Euler angles set to -90 degrees, 0 degrees, and -90 degrees, and the translation vector set to 0, 0, and 0. For the first face, draw the first loop by starting a line from the origin to the 0.005, 0, then draw a line to 0.5, 0.625, followed by a line to 0.25, 0.625. Continue with a line to 0.25, 0.25, then to 0, 0.25, and finally back to the origin. Scale the entire sketch by a factor of 0.625. Rotate the scaled sketch using the same Euler a

Original dataset: indices 20 and 15 are most complex

In [ ]:
def gpt(system_prompt: str, user_prompt: str, model: str) -> str:
    client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model=model,
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

def open_router(system_prompt: str, user_prompt: str, model: str) -> str:
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key = os.getenv('OPENROUTER_API_KEY')
    )
    response = client.chat.completions.create(
        model=model,
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [ ]:
models_orouter = ['google/gemini-2.5-pro', 'x-ai/grok-4', 'openai/gpt-4.1']
models_gpt = ['gpt-4.1-mini','gpt-4.1','o3']
# idx = 20
# code, description = dataset[idx]["code"], dataset[idx]["description"]
prompt = {
    'pipe': 'It is a pipe of length 100 mm, inner radius is 10 mm, and outer radius is 20 mm.',
    'gear': 'It is a gear wheel which looks like a cylinder with height 25 mm and radius of 25 mm and a hole of radius 10 mm. There is a 6 cogs uniformly located on the cylinder covering full cylinder depth, each cog is trapezoidal with long base such that the long bases of all 6 trapezoids covers the cylinder around, short base of size 5 mm.',
    'tray': 'It is a parallelepiped of length 100 mm, width 50 mm, and height 20 mm. There is a square recess of size 70x30 mm and depth 15 mm in the middle. The recess is offsetted to the right shorter edge of parallelepiped by 10 mm.',
    'ladder': 'It is a two parallelepipeds, one staying on other. The foundation parallelepiped has width 200 mm, height 20 mm, and depth 50 mm. It is truncated on the whole depth and height from the right side with angle 45 degrees. The upper parallelepiped is staying on the foundation edge and had 200 mm width, 25 mm height and 25 mm depth.',
    'support': 'The figure consists of foundation of size 200x50x20 mm, there is another parallelepiped staying on foundation in the middle with size 75x50x50 mm. Upper parallelepiped is cutted out by the cylinder of radius 50 mm and height 50 mm. There is also a two holes of radius 10 mm on the foundation in the middle between edges and the second parallelepiped.',
}
description = prompt['gear']

In [212]:
system_prompt = '''You are an expert in parametric 3D modeling using CadQuery and Python. Your task is to write a Python code using the CadQuery library that generates a 3D model matching a reference information about the shape as closely as possible.

Requirements:
1. The code must use CadQuery primitives and operations.
2. The code should assign final CadQuery solid object (cq.Workplane with 3D geometry) to variable 'r'.
3. The script must be executable in a standard Python environment with CadQuery installed (no other packages).
4. Do not include any comments  to the code.

You have a text shape description with important information.'''

user_prompt = f'''Shape description:
{description}'''

In [213]:
for i in range(5):
    result = gpt(system_prompt, user_prompt, models_gpt[1])
    result = result.replace('```python', '').replace('```', '')
    obj, std_out = execute_cadquery_code(result)
    
    if obj is None:
        print('error!')
        user_prompt = f'''Shape description:
        {description}

        Your code returned error: {std_out}. Please correct the code.
        Code: {result}'''
    else:
        break

display(obj)